## Detekcia meteorov z obrázkov kamier AMOS pomocou hlbokého učenia
### Príprava pracovného prostredia pre spustenie YOLOv5
Naklonovanie github repozitára yolov5 a spustenie inštalácie pre potrebné knižnice

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install opencv-contrib-python
!pip install cv2

%cd yolov5
pip install -r requirements.txt
%cd ..

Import všetkých potrebných knižníc

In [ ]:
import PyPDF2
import os
import tempfile
import zipfile,fnmatch
import csv
import pandas as pd
import numpy as np
import json
import math 
import cv2
import shutil
import operator
from PIL import Image, ImageDraw

## Vytvorenie jednotlivých množín

### Rozdelenie obrázkov na trénovaciu, validačnú a testovaciu množinu

Importovanie nástroju pre automatické rozdelenie obrázkov zo súboru `dataloaders.py` (používateľ sa musí nachádzať v priečinku `yolov5`, ktorý obsahuje priečinok `utils`, následne sa vráti späť)

In [ ]:
%cd yolov5
import torch
from utils.dataloaders import *
%cd ..

autosplit('meteor4000/images', weights=(0.85, 0.1, 0.05))

Rozdelenie obrázkov v datasete na trénovaciu, validačnú a testovaciu množinu v pomere `85:10:5`

In [ ]:
autosplit('meteor4000/images', weights=(0.85, 0.1, 0.05))

Po tomto kroku sa v priečinku meteor4000 objavia súbory `autosplit_train.txt`, `autosplit_val.txt` a `autosplit_test.txt`, ktoré si premenujeme na `train.txt`, `val.txt` a `test.txt`

## Vytvorenie YAML súboru datasetu

#### vnútri .yaml súboru je štruktúra nasledovná:

`train: ../meteor4000/train.txt`  - cesta k .txt súboru s trénovacími obrázkami

`val: ../meteor4000/val.txt`  - cesta k .txt súboru s validačnými obrázkami

`test: ../meteor4000/test.txt` - cesta k .txt súboru s testovacími obrázkami

`nc: 2` - počet tried

`names: ['meteor', 'satellite']` - názvy tried

## Spustenie trénovania modelu

Cesty sú relatívne ku priečinku, v ktorom sa práve nachádzame

In [1]:
#!python yolov5/train.py --img 1280 --rect --batch 15 --epochs 300 --data meteor4000/meteor4000.yaml --weights yolov5/yolov5s.pt --save-period 10 --patience 20

## Spustenie validácie modelu

Cesty sú relatíve ku priečinku, v ktorom sa práve nachádzame

In [ ]:
!python yolov5/val.py --weights model/meteor4000.pt --data meteor4000/meteor4000.yaml --img 1280

## Spustenie detekcie
Cesty sú relatíve ku priečinku, v ktorom sa práve nachádzame

In [ ]:
!python yolov5/detect.py --weights model/meteor4000.pt --source test --iou 0.45 --conf 0.3 --img 1280 --save-txt --save-conf

## Spracovanie výstupov YOLOv5

Vytvorenie tabuľky so všetkými detekciami a jej export do súboru `output.csv`\
directory - cesta ku priečinku s textovými súbormi z detekcie, predvolene v priečinku `yolov5/runs/detect/expXX`

In [ ]:
directory = r"exp14/labels" #folder with .txt labels from detection, default in yolov5/runs/detect/expXX
Detections = pd.DataFrame(columns=['file','class','confidence', 'x', 'y', 'width', 'height'])
#mCounter = 0 #counter for meteor detections
#sCounter = 0 #counter for satellite detections

for file_name in os.listdir(directory):
        if file_name.endswith(".txt"):
            file_path = os.path.join(directory, file_name)
                                    
        with open(file_path, "r") as file:
            for line in file:
                line = file_name[:-3] + "jpg " + line
                values = line.strip().split()
                if len(values) == 7:
                    if values[1] == "0": 
                        values[1] = "meteor"
                        #mCounter+=1
                    else: 
                        values[1] = "satellite"
                        #sCounter+=1
                    values[6] = round(float(values[6]),2)
                    detection = pd.DataFrame({
                         'file': [values[0]],
                         'class': [values[1]],
                         'confidence': [values[6]],
                         'x': [values[2]],
                         'y': [values[3]],
                         'width': [values[4]],
                         'height': [values[5]]})
                    Detections = pd.concat([Detections, detection], ignore_index=True)
                
#print("Detected " + str(mCounter) + " meteors and " + str(sCounter) + " satellites")
#print("\nDetections:")
#print(Detections)
Detections.to_csv("output.csv")